In [21]:
import os
reps=[]
for code in ['510330','159915','510300','510310','510500','159919']:
    report = pd.concat([pd.read_csv(f) for f in os.listdir() if 'Result_{}'.format(code) in f and '.csv' in f]).set_index('name').drop_duplicates()
    report['策略-基准盈亏']=report['策略浮动盈亏']-report['基准浮动盈亏']
    report['CODE']=code
    reps.append(report)

In [22]:
report=pd.concat(reps)
reps=[]

In [46]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val > 0 else 'black'
    return 'color: %s' % color

以下选取均值和中位数均大于0的数据进行说明：

* `159915` 完整情况下中位数小于均值，说明更多的数据集中在均值左侧，实际操作中可能无法达到均值；
* `510310` 拆分情况下中位数大于均值，说明更多的数据集中在均值右侧，实际操作中可能会超越均值；
* `510500` 拆分情况下中位数小于均值，说明更多的数据集中在均值左侧，实际操作中可能无法达到均值；**中位数接近0**，盈利可能性较低；

In [47]:
report.groupby(['CODE','name'])['策略-基准盈亏'].describe().style.applymap(color_negative_red)

In [65]:
backreturn=report.groupby(['CODE','name'])['基准最大回撤','策略最大回撤'].agg(['mean','median'])
backreturn

基准最大回撤          策略最大回撤        
               mean  median    mean  median
CODE   name                                
159915 完整   -0.2332 -0.2007 -0.1542 -0.1195
       拆分   -0.2322 -0.2007 -0.1657 -0.1280
159919 完整   -0.1654 -0.1311 -0.1063 -0.0793
       拆分   -0.1697 -0.1311 -0.1161 -0.0894
510300 完整   -0.1654 -0.1311 -0.1045 -0.0843
       拆分   -0.1660 -0.1311 -0.1037 -0.1013
510310 完整   -0.2334 -0.2007 -0.1146 -0.0893
       拆分   -0.2404 -0.2007 -0.0878 -0.0773
510330 完整   -0.1654 -0.1311 -0.1236 -0.0880
       拆分   -0.1657 -0.1311 -0.1129 -0.0952
510500 完整   -0.2133 -0.1699 -0.1769 -0.1351
       拆分   -0.2144 -0.1699 -0.1738 -0.1373

`510310`使用策略后最大回撤相对于基准会降低`10%+`；

In [67]:
(backreturn['策略最大回撤']-backreturn['基准最大回撤']).style.bar(color='#d65f5f')

In [54]:
pd.set_option('display.max_rows', 500)

def win_rate_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val > 0.5 else 'black'
    return 'color: %s' % color

只有`159915`和`510310`两支无论持否拆分，跑赢基准的可能性均高于`50%`；但是这两只的情况又刚好相反，`510310`是拆分后跑赢基准较高，最高为回看3年的情况下能够达到`80.39%`，而`159915`则是非拆分时跑赢基准的可能性较高，无论回看几年大约在`65%`左右；

In [55]:
import matplotlib.ticker as mtick
win_rate = report.groupby(
    ['CODE','name', 'passyears'])['跑赢基准'].value_counts(normalize=True).to_frame()
win_rate[win_rate.index.get_level_values(3) == 1].style.format("{:.2%}").applymap(win_rate_red)

In [56]:
import matplotlib.ticker as mtick
win_rate = report.groupby(
    ['CODE','name', 'passyears'])['是否盈利'].value_counts(normalize=True).to_frame()
win_rate[win_rate.index.get_level_values(3) == 1].style.format("{:.2%}").applymap(win_rate_red)